In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from math import *
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

### Scrap Data

In [10]:
class Bot:

    def ScrapHouses(self, MaxRetrieves):

        URL_1 = 'https://www.century21global.com/for-sale-residential/USA/NY/New-York-Metro-Area?pageNo='
        page = 1
        pageIncrement = 20
        maxRetrieves = MaxRetrieves
        Wait = 30
        Houses = []
        House_infos = []

        url = URL_1 + str(page)

        options = Options()
        options.headless = True   # Do we want to see the website ? True = no; False = yes
        options.add_experimental_option("detach", True)

        browser = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        browser.maximize_window()
        browser.get(url)
        browser.set_page_load_timeout(Wait)

        Houses_to_scrap_string = browser.find_element_by_xpath('//*[@id="propertySearchResults"]/div[1]/div/div[2]/span[2]')
        Houses_to_scrap = int(Houses_to_scrap_string.text.replace('(', '').replace(')','').replace(' ','').replace(',',''))

        nb = 1
        count = 1


        while True :

            try :
                if nb > min(maxRetrieves,Houses_to_scrap) :
                    break

                if count > pageIncrement :
                    count = 1
                    page += 1

                url = URL_1 + str(page)
                browser.get(url)
                browser.set_page_load_timeout(Wait)

                house = browser.find_element_by_xpath('//*[@id="propertySearchResults"]/div[' + str(count+2) + ']')
                house.click()
                browser.set_page_load_timeout(Wait)

                # Get houses information

                price = browser.find_element_by_xpath('//*[@id="listingDetails"]/div[2]/div/div/h2').text

                location = browser.find_element_by_xpath('//*[@id="listingDetails"]/div[2]/div/div/div[2]').text.replace('\n',' /;/ ')

                details_1 = []
                details_2 = []

                d_1 = browser.find_element_by_xpath('//*[@id="listingDetails"]/div[2]/div/div/div[5]')
                for x in d_1.find_elements_by_tag_name('div') :
                    if x.text != '' :
                        details_1.append(x.text)


                d_2 = browser.find_element_by_xpath('//*[@id="listingDetails"]/div[2]/div/div/ul')
                for x in d_2.find_elements_by_tag_name('li') :
                    if x.text != '' :
                        details_2.append(x.text)

                House_infos = {
                    'Price' : price,
                    'Location' : location,
                    'Caracteristics_1' : details_1,
                    'Caracteristics_2' : details_2
                }

                Houses.append(House_infos)

                url = URL_1 + str(page)
                browser.get(url)
                browser.set_page_load_timeout(Wait)

                count += 1

                print(str(nb) + '/' + str(min(Houses_to_scrap,maxRetrieves)))
                nb += 1
            
            except Exception as e :
                
                try :

                    print("Exception",e)

                    url = URL_1 + str(page)
                    browser.get(url)
                    browser.set_page_load_timeout(Wait)

                    count += 1

                    print(str(nb) + '/' + str(min(Houses_to_scrap,maxRetrieves)))
                    nb += 1

                except Exception as e :

                    print("Exception",e)
                    break

        return Houses


In [ ]:
Scrapper = Bot()
Results = Scrapper.ScrapHouses(10000)
DF = pd.DataFrame(Results)
DF.to_excel(r'C:\Users\lohue\Desktop\Real_Estate Valuation model\Outputs_NYC.xlsx', index = False)

### Clean Data

In [2]:
NY = pd.read_excel('Outputs_NYC.xlsx').dropna()

def search_string(DF,column_,search_sing, search_plurial) :

    x = []

    for i in range(0, len(DF)):
            if DF[column_][i].find(search_sing) == -1 and DF[column_][i].find(search_plurial) == -1:
                x.append('na')
            else:
                if DF[column_][i].find(search_sing) != -1 :
                    j = 3
                    while DF[column_][i][DF[column_][i].find(search_sing)-j] != "'" and DF[column_][i][DF[column_][i].find(search_sing)-j] != '"':
                        j = j+1
                    x.append(DF[column_][i][DF[column_][i].find(search_sing)-j+1:DF[column_][i].find(search_sing)-1])
                else :
                    j = 3
                    while DF[column_][i][DF[column_][i].find(search_plurial)-j] != "'" and DF[column_][i][DF[column_][i].find(search_plurial)-j] != '"':
                        j = j+1
                    x.append(DF[column_][i][DF[column_][i].find(search_plurial)-j+1:DF[column_][i].find(search_plurial)-1])

    return x


def Construction_year(DF, column_) :

    x = []

    for i in range(0, len(DF)):
        if DF[column_][i].find("Year Built:") != -1 :
            j = len("Year Built:")+1
            while DF[column_][i][DF[column_][i].find("Year Built:")+j] != "'" :
                j += 1
            x.append(int(DF[column_][i][DF[column_][i].find("Year Built:")+len("Year Built:")+1:DF[column_][i].find("Year Built:")+j]))
        elif DF[column_][i].find("New Construction") != -1 :
            x.append(2021)
        else :
            x.append('na')

    return x



def Data_cleaning(DF, city) :

    DF.insert(1, "Currency", DF['Price'].str[-3:], True)
    DF['Price'] = pd.to_numeric(DF['Price'].str.replace(DF['Price'][0][-4:], '').str.replace('$','').str.replace(' ','').str.replace(',',''))
    DF['Location'], DF['Ville']  = DF['Location'].str.split('/;/', 1).str
    DF['Location'] = DF['Location'].str.replace(', '+ city,'')
    DF['Ville'], DF['Pays'] = DF['Ville'].str.split('/;/', 1).str
    for i in range(0, len(DF)):
        if DF['Ville'][i].find(',') == -1:
            DF['Ville'][i] = DF['Ville'][i][0:-6]
        else :
            DF['Ville'][i] = DF['Ville'][i][0:DF['Ville'][i].find(',')]
        
    DF = DF.drop(['Location'],axis = 1)
    DF['Bedrooms'] = search_string(DF,'Caracteristics_1','bedroom', 'bedrooms')
    DF['Bathrooms'] = search_string(DF,'Caracteristics_1','full bath', 'full baths')
    DF['Area'] = search_string(DF,'Caracteristics_1','m2', 'm2')
    DF['Area'] = DF['Area'].str.replace('.','')
    DF['Construction year'] = Construction_year(DF, 'Caracteristics_2')

    DF = DF.drop(DF[DF['Ville'] == 'na'].index | DF[DF['Bedrooms'] == 'na'].index | DF[DF['Bathrooms'] == 'na'].index | DF[DF['Area'] == 'na'].index | DF[DF['Construction year'] == 'na'].index).reset_index()

    DF['Bedrooms'] = pd.to_numeric(
        DF['Bedrooms'], 
        errors='coerce'
        ).fillna(0).astype('int')

    DF['Bathrooms'] = pd.to_numeric(
        DF['Bathrooms'], 
        errors='coerce'
        ).fillna(0).astype('int')

    DF['Area'] = pd.to_numeric(
        DF['Area'], 
        errors='coerce'
        ).fillna(0).astype('int')

    DF['Construction year'] = pd.to_numeric(
        DF['Construction year'], 
        errors='coerce'
        ).fillna(0).astype('int')

    return DF


In [3]:
NY = pd.read_excel('Outputs_NYC.xlsx').dropna()


df = Data_cleaning(NY, 'New York')
df.to_excel(r'C:\Users\lohue\Desktop\Real_Estate Valuation model\Outputs_clean.xlsx', index = False)


C:\Users\lohue\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lohue\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Model

In [10]:
NY_dummy = []
for i in range(0, len(df.Ville)):
    if df.Ville[i] == ' New York ' :
        NY_dummy.append(1)
    else :
        NY_dummy.append(0)

df['NY'] = NY_dummy

#Let's select some features that we want to use for regression.

cdf = df[['Price','Bedrooms','Bathrooms','Area','Construction year','NY']]

# Let's split the data into 2 samples 

msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

cdf.head(10)

,Price,Bedrooms,Bathrooms,Area,Construction year,NY
0,7395000,4,4,237174,2021,1
1,6100000,4,4,246649,2021,1
2,5900000,3,3,225004,2021,1
3,5250000,3,3,210233,2021,1
4,3990000,2,2,155886,2021,1
5,3799999,5,4,318833,1930,0
6,3710000,2,2,15858,2021,1
7,3690000,2,2,155515,2021,1
8,3595000,3,3,151427,2021,1
9,3495000,3,3,151427,2021,1


In [8]:
# Multiple Regression Model

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

regr = linear_model.LinearRegression()

x = np.asanyarray(train[['Bedrooms','Bathrooms','Area','Construction year','NY']])
y = np.asanyarray(train[['Price']])
regr.fit (x, y)
# The coefficients
print ('Coefficients: ', regr.coef_)

Coefficients:  [[1.16784503e+04 2.58333560e+05 3.00499752e-01 8.32586787e+02
  2.94351428e+06]]


In [9]:
# Prediction


y_hat= regr.predict(test[['Bedrooms','Bathrooms','Area','Construction year','NY']])
x = np.asanyarray(test[['Bedrooms','Bathrooms','Area','Construction year','NY']])
y = np.asanyarray(test[['Price']])
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))


Residual sum of squares: 189723742040.35
Variance score: 0.71
